# **KLASIFIKASI EMOSI menggunakan metode K-Nearest Neighbor**

# **TUJUAN ANALISIS**

1. Klasifikasi Emosi berdasarkan rekaman suara

# **Deskripsi Data**

dataset diambil dari : https://www.kaggle.com/datasets/ejlok1/toronto-emotional-speech-set-tess?resource=download
Dataset audio ini akan di ektraksi menjadi beberapa fitur numeric ZCR dan RMSE.
Dataset ini berisi serangkaian 200 kata target yang diucapkan dalam frasa pembawa "Katakan kata _" oleh dua aktris (berusia 26 dan 64 tahun), dan rekaman dibuat untuk setiap kata dengan menggambarkan tujuh emosi berbeda (marah, jijik, takut, bahagia, kaget menyenangkan, sedih, dan netral). Totalnya terdapat 2800 titik data (file audio) dalam dataset ini.

Penyusunan dataset ini dilakukan dengan cara bahwa setiap dari dua aktris perempuan dan emosi yang mereka tunjukkan terkandung dalam folder tersendiri. Di dalamnya, terdapat file audio untuk semua 200 kata target. Format file audio ini adalah format WAV.

Dengan kata lain, dataset ini memberikan akses ke rekaman suara dari dua aktris perempuan yang berbeda usia, masing-masing mengucapkan 200 kata target dalam tujuh berbagai emosi. Setiap kombinasi kata dan emosi memiliki rekaman suara sendiri-sendiri, dan semuanya tersusun dengan baik dalam struktur folder yang jelas. Format file audio yang digunakan adalah WAV.



### **Menghitung statistik sinyal yang akan dijadikan fitur**

1. **Zero Crossing Rate (ZCR):** <br>
   Zero Crossing Rate (ZCR) adalah metrik yang mengukur seberapa sering sinyal audio melintasi nilai nol per detik. Ini memberikan wawasan tentang perubahan arah atau fluktuasi dalam gelombang suara. Komponen ZCR mencakup: <br>
   - *zcr_mean:* Nilai rata-rata ZCR, mencerminkan tingkat perubahan rata-rata dalam sinyal. <br>
   - *zcr_median:* Nilai median ZCR, merupakan nilai tengah dalam distribusi ZCR. <br>
   - *zcr_std_dev:* Deviasi standar dari ZCR, mengukur sejauh mana nilai ZCR tersebar dari nilai rata-rata. <br>
   - *zcr_kurtosis:* Kurtosis dari distribusi ZCR, memberikan informasi tentang kekontrasan antara ekor dan puncak distribusi. <br>
   - *zcr_skew:* Skewness dari distribusi ZCR, mengindikasikan asimetri distribusi ZCR. <br>
2. **Root Mean Square Error (RMSE):** <br>
   Root Mean Square Error (RMSE) adalah metrik evaluasi yang mengukur rata-rata perbedaan antara nilai aktual dan nilai yang diprediksi, dalam konteks ini, terkait dengan amplitudo sinyal audio. Komponen RMSE melibatkan: <br>
   - *rmse:* Nilai RMSE dari sinyal, memberikan ukuran kesalahan total antara nilai aktual dan nilai prediksi. <br>
   - *rmse_median:* Nilai median dari RMSE, menunjukkan nilai tengah dalam distribusi RMSE. <br>
   - *rmse_std_dev:* Deviasi standar dari RMSE, mengukur sebaran nilai RMSE dari nilai rata-rata. <br>
   - *rmse_kurtosis:* Kurtosis dari distribusi RMSE, memberikan informasi tentang bentuk dan karakteristik distribusi kesalahan. <br>
   - *rmse_skew:* Skewness dari distribusi RMSE, mengindikasikan asimetri distribusi RMSE. <br>

   **RUMUS :**<br>
   <br>
   $\text{{Mean}} = \frac{1}{n} \sum_{i=1}^{n} y_i$ <br>
   <br>
  $\text{{Median}} = \begin{cases}
      y_{(\frac{n}{2})} & \text{{if }} n \text{{ is odd}} \\
      \frac{1}{2} (y_{(\frac{n}{2})} + y_{(\frac{n}{2} + 1)}) & \text{{if }} n \text{{ is even}}
   \end{cases}$ <br>
<br>
  $\text{{Standard Deviasi}} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^2}$ <br>
<br>
  $\text{{Kurtosis}} = \frac{\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^4}{\left(\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^2\right)^2} - 3$ <br>
<br>
  $\text{{Skewness}} = \frac{\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^3}{\left(\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^2\right)^{3/2}}$ <br>
<br>

# **PRE-PROCESSING**

sebelum melakukan analisis data kita akan melakukan ekstraksi audio menjadi beberapa fitur numerik dan akan dilakukan standarisasi data dalam rentang 0-1 hal ini di lakukan di pre-processing

## **EKSTRAKSI AUDIO**

In [ ]:
pip install librosa

**Menghubungkan ke google drive**

In [ ]:
# Import library drive dari modul google.colab untuk mengakses Google Drive
from google.colab import drive
# Mount Google Drive ke dalam direktori /content/drive di Google Colab
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/PSD/emosi

/content/drive/MyDrive/PSD/emosi


In [ ]:
# Import library untuk berinteraksi dengan sistem operasi (OS)
import os
# Import pustaka librosa untuk analisis sinyal suara
import librosa
# Import pustaka numpy untuk manipulasi data numerik
import numpy as np
# Import pustaka pandas untuk analisis dan manipulasi data tabular
import pandas as pd
# Import beberapa fungsi statistik dari pustaka scipy
from scipy.stats import skew, kurtosis, mode

In [ ]:
#variael yang menyimpan list nama folder yang berisi dataset emosi
folders=['YAF_sad','YAF_pleasant_surprised','YAF_neutral',
         'YAF_happy','YAF_fear','YAF_disgust','YAF_angry',
         'OAF_sad','OAF_pleasant_surprise','OAF_neutral',
         'OAF_happy','OAF_fear','OAF_disgust',
         'OAF_angry',
         ]

**Ektraksi Audio menjadi fitur numerik**

In [ ]:
def calculate_statistics(audio_path):
    # Menggunakan librosa untuk memuat file audio
    y, sr = librosa.load(audio_path)

    # MENGHITUNG NILAI ZERO-CROSSING RATE (ZCR)
    zcr_mean = np.mean(librosa.feature.zero_crossing_rate(y=y))
    zcr_median = np.median(librosa.feature.zero_crossing_rate(y=y))
    zcr_std_dev = np.std(librosa.feature.zero_crossing_rate(y=y))
    zcr_kurtosis = kurtosis(librosa.feature.zero_crossing_rate(y=y)[0])
    zcr_skew = skew(librosa.feature.zero_crossing_rate(y=y)[0])

    # MENGHITUNG NILAI ROOT MEAN SQUARE ERROR (RMSE)
    rmse = np.sum(y**2) / len(y)
    rmse_median = np.median(y**2)
    rmse_std_dev = np.std(y**2)
    rmse_kurtosis = kurtosis(y**2)
    rmse_skew = skew(y**2)

    # return daftar nilai statistik
    return [zcr_mean, zcr_median, zcr_std_dev, zcr_kurtosis, zcr_skew, rmse, rmse_median, rmse_std_dev, rmse_kurtosis, rmse_skew]

In [ ]:
features =[] #variabel yang akan digunakan untuk fitur

In [ ]:
# Loop melalui setiap folder dalam daftar 'folders'
for folder in folders:
    # Membuat path lengkap untuk folder saat ini
    folder_path = f'{folder}'

    # Loop melalui setiap file dalam folder saat ini
    for filename in os.listdir(folder_path):

        # Memeriksa apakah file berakhir dengan ekstensi '.wav'
        if filename.endswith('.wav'):

            # Membuat path lengkap untuk file audio saat ini
            audio_path = os.path.join(folder_path, filename)

            # Menghitung statistik dari file audio dan menyimpannya dalam variabel 'statistics'
            statistics = calculate_statistics(audio_path)

            # Menambahkan label folder dan statistik audio ke dalam daftar fitur 'features'
            features.append([folder] + statistics)

In [ ]:
columns =  ['Label']+['ZCR Mean', 'ZCR Median', 'ZCR Std Dev', 'ZCR Kurtosis', 'ZCR Skew', 'RMSE', 'RMSE Median', 'RMSE Std Dev', 'RMSE Kurtosis', 'RMSE Skew']
df = pd.DataFrame(features, columns=columns)

In [ ]:
# Menampilkan file CSV
df

,Label,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew,RMSE,RMSE Median,RMSE Std Dev,RMSE Kurtosis,RMSE Skew
0,YAF_sad,0.128581,0.033203,0.225249,3.231511,2.213080,0.001907,0.000391,0.004361,28.695116,4.718993
1,YAF_sad,0.118958,0.028320,0.195816,4.225416,2.337965,0.000546,0.000059,0.001320,37.525358,5.243677
2,YAF_sad,0.124418,0.030762,0.203419,3.494584,2.197851,0.000502,0.000045,0.001103,20.908828,4.065221
3,YAF_sad,0.105494,0.035156,0.194269,5.715395,2.682275,0.001279,0.000270,0.002408,15.520610,3.484518
4,YAF_sad,0.138011,0.057129,0.193587,3.628069,2.167333,0.001019,0.000084,0.002397,23.264337,4.194062
...,...,...,...,...,...,...,...,...,...,...,...
2801,OAF_angry,0.083810,0.052246,0.099268,4.521899,2.239367,0.001529,0.000058,0.003707,21.352151,4.054795
2802,OAF_angry,0.115001,0.060059,0.118703,3.370580,2.012270,0.001743,0.000146,0.004667,41.713213,5.561090
2803,OAF_angry,0.083991,0.051270,0.091742,7.208814,2.744011,0.003030,0.000291,0.006712,21.150835,3.983283
2804,OAF_angry,0.085005,0.059082,0.090037,5.366187,2.422199,0.001496,0.000135,0.004290,31.532767,5.091522


1. **Label** <br>
  - OAF_angry: Ekspresi marah dari pembicara OAF. <br>
  - OAF_disgust: Ekspresi jijik atau muak dari pembicara OAF. <br>
  - OAF_Fear: Ekspresi takut dari pembicara OAF. <br>
  - OAF_happy: Ekspresi bahagia dari pembicara OAF. <br>
  - OAF_neutral: Ekspresi netral dari pembicara OAF. <br>
  - OAF_Pleasant_surprise: Ekspresi kejutan menyenangkan dari pembicara OAF. <br>
  - OAF_Sad: Ekspresi sedih dari pembicara OAF. <br>
  - YAF_angry: Ekspresi marah dari pembicara YAF. <br>
  - YAF_disgust: Ekspresi jijik atau muak dari pembicara YAF. <br>
  - YAF_fear: Ekspresi takut dari pembicara YAF. <br>
  - YAF_happy: Ekspresi bahagia dari pembicara YAF. <br>
  - YAF_neutral: Ekspresi netral dari pembicara YAF. <br>
  - YAF_Pleasant_surprise: Ekspresi kejutan menyenangkan dari pembicara YAF. <br>
  - YAF_sad: Ekspresi sedih dari pembicara YAF. <br>

In [ ]:
#menyimpan file hasil dari ekstraksi fitur
df.to_csv('dataemosi.csv',index=False)

# **NORMALISASI STANDARD SCALER**

StandardScaler adalah suatu metode dalam pemrosesan data yang digunakan untuk mentransformasi fitur-fitur data sehingga memiliki mean 0 dan deviasi standar 1. Ini adalah salah satu langkah umum dalam pra-pemrosesan data sebelum membangun model. Proses ini membantu menghilangkan perbedaan skala antar fitur, sehingga setiap fitur memiliki pengaruh yang sebanding pada model.


Rumus Standard Scaler: <br>
$z = \frac{{x - \mu}}{{\sigma}}$

di mana:<br>
- z adalah nilai yang telah di-standarisasi,<br>
- x adalah nilai asli,<br>
- μ adalah rata-rata dari sampel,<br>
- σ adalah deviasi standar dari sampel.<br>


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pickle import dump

In [ ]:
# Baca data dari file CSV
dataknn = pd.read_csv('dataemosi.csv')

# Pisahkan fitur (X) dan label (y)
X = dataknn.drop(['Label'], axis=1)  # Menghapus kolom 'Label' sebagai fitur dan menyimpannya di variabel X
y = dataknn['Label']  # Mengambil kolom 'Label' sebagai label dan menyimpannya di variabel y

# Bagi data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2)

# Tentukan scaler
scaler = StandardScaler()

# Terapkan scaler pada dataset pelatihan
scaler.fit(X_train)

# Simpan scaler
dump(scaler, open('scaler.pkl', 'wb'))

# Normalisasi dataset pelatihan
X_train_scaled = scaler.transform(X_train)

# Tampilkan datasetknn
dataknn


,Label,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew,RMSE,RMSE Median,RMSE Std Dev,RMSE Kurtosis,RMSE Skew
0,YAF_sad,0.128581,0.033203,0.225249,3.231511,2.213080,0.001907,0.000391,0.004361,28.695116,4.718993
1,YAF_sad,0.118958,0.028320,0.195816,4.225416,2.337965,0.000546,0.000059,0.001320,37.525358,5.243677
2,YAF_sad,0.124418,0.030762,0.203419,3.494584,2.197851,0.000502,0.000045,0.001103,20.908828,4.065221
3,YAF_sad,0.105494,0.035156,0.194269,5.715395,2.682275,0.001279,0.000270,0.002408,15.520610,3.484518
4,YAF_sad,0.138011,0.057129,0.193587,3.628069,2.167333,0.001019,0.000084,0.002397,23.264337,4.194062
...,...,...,...,...,...,...,...,...,...,...,...
2801,OAF_angry,0.083810,0.052246,0.099268,4.521899,2.239367,0.001529,0.000058,0.003707,21.352151,4.054795
2802,OAF_angry,0.115001,0.060059,0.118703,3.370580,2.012270,0.001743,0.000146,0.004667,41.713213,5.561090
2803,OAF_angry,0.083991,0.051270,0.091742,7.208814,2.744011,0.003030,0.000291,0.006712,21.150835,3.983283
2804,OAF_angry,0.085005,0.059082,0.090037,5.366187,2.422199,0.001496,0.000135,0.004290,31.532767,5.091522


In [ ]:
import pickle
# Membuka file 'scaler.pkl' untuk membaca (mode 'rb' untuk mode baca biner)
with open('scaler.pkl', 'rb') as standarisasi:
    loadscal = pickle.load(standarisasi)


In [ ]:
X_test_scaled=loadscal.transform(X_test) #normalisasi X testing dari hasil normalisasi X train yang disimpan dalam model

# **PEMODELAN**


**K-Nearest Neighbors (KNN):**<br>

**Konsep:**<br>
K-Nearest Neighbors (KNN) adalah algoritma pembelajaran mesin sederhana yang digunakan untuk klasifikasi dan regresi. Konsep dasar KNN adalah memprediksi label suatu sampel berdasarkan mayoritas label dari K tetangga terdekatnya.

**Langkah-langkah KNN:**<br>

1.Menentukan Jarak:<br> Hitung jarak (biasanya Euclidean) antara sampel yang akan diprediksi dengan semua sampel dalam set pelatihan.<br>
2. Memilih Tetangga:<br> Pilih K tetangga terdekat berdasarkan jarak yang dihitung.<br>
3. Menentukan Mayoritas:<br> Untuk klasifikasi, atribut target dari sampel baru diberikan label yang paling umum di antara K tetangga tersebut. Untuk regresi, nilai target dari sampel baru dihitung sebagai rata-rata atau median dari nilai target K tetangga tersebut.<br>
**Parameter Utama:**<br>
Nilai K: <br>Jumlah tetangga terdekat yang akan dipertimbangkan. Nilai K yang tepat dapat mempengaruhi kinerja algoritma.<br>
**Rumus:** <br>
Rumus mencari jarak dengan euclidean : <br>
$\text{Euclidean Distance} = \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}$ <br>
**Rumus Klasifikasi KNN :** <br>
$\hat{y} = \text{majority}(\{y_i \,|\, (x_i, y_i) \in \text{k-nearest neighbors of } x\})$

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pickle import dump
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
K = 30
acc = np.zeros((K-1))
# Loop untuk mencari nilai akurasi dengan jumlah tetangga (k) dari 1 hingga K dengan langkah 2
for n in range(1, K, 2):
    # Membuat model k-NN dengan nilai k tertentu dan metrik euclidean
    knn = KNeighborsClassifier(n_neighbors=n, metric="euclidean").fit(X_train_scaled, y_train)

    # Melakukan prediksi pada data uji
    y_pred = knn.predict(X_test_scaled)

    # Menghitung dan menyimpan akurasi untuk nilai k saat ini
    acc[n-1] = accuracy_score(y_test, y_pred)

# Menampilkan nilai akurasi terbaik dan nilai k yang sesuai
print('Akurasi terbaik adalah', acc.max(), 'dengan nilai k =', acc.argmax() + 1)


Akurasi terbaik adalah 0.7206405693950177 dengan nilai k = 13


In [ ]:
clf = KNeighborsClassifier(n_neighbors= 13, metric = "euclidean")

In [ ]:
clf.fit(X_train_scaled, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=13)

In [ ]:
# Melakukan prediksi pada data uji yang telah dinormalisasi menggunakan model RandomForestClassifier
y_pred = clf.predict(X_test_scaled)

# Menampilkan hasil prediksi
y_pred

array(['YAF_fear', 'YAF_pleasant_surprised', 'YAF_fear', 'YAF_angry',
       'YAF_angry', 'OAF_disgust', 'OAF_disgust', 'YAF_neutral',
       'OAF_disgust', 'OAF_neutral', 'YAF_fear', 'YAF_happy',
       'YAF_pleasant_surprised', 'OAF_happy', 'YAF_pleasant_surprised',
       'YAF_pleasant_surprised', 'OAF_fear', 'YAF_sad', 'YAF_angry',
       'YAF_pleasant_surprised', 'YAF_disgust', 'OAF_neutral',
       'YAF_neutral', 'OAF_pleasant_surprise', 'YAF_sad', 'OAF_disgust',
       'OAF_disgust', 'OAF_angry', 'YAF_pleasant_surprised', 'YAF_angry',
       'YAF_happy', 'OAF_happy', 'OAF_sad', 'OAF_fear', 'YAF_disgust',
       'OAF_happy', 'OAF_disgust', 'OAF_neutral',
       'YAF_pleasant_surprised', 'YAF_pleasant_surprised', 'OAF_sad',
       'OAF_happy', 'YAF_neutral', 'OAF_fear', 'OAF_fear', 'YAF_disgust',
       'YAF_sad', 'OAF_pleasant_surprise', 'YAF_disgust', 'OAF_disgust',
       'OAF_sad', 'OAF_happy', 'YAF_disgust', 'YAF_fear', 'YAF_disgust',
       'OAF_neutral', 'OAF_angry', 'YAF_n

In [ ]:
# Menghitung akurasi antara nilai sebenarnya dan nilai prediksi
accuracy = accuracy_score(y_test, y_pred)
# Menampilkan nilai akurasi
print("Akurasi:", accuracy)

Akurasi: 0.7206405693950177


In [ ]:
dump(clf, open('knnfix.pkl', 'wb'))

# **PCA**


Principal Component Analysis (PCA) adalah suatu metode dalam statistika multivariat yang digunakan untuk mengubah dataset yang kompleks menjadi bentuk yang lebih sederhana dan lebih mudah diinterpretasikan. Tujuan utama dari PCA adalah mereduksi dimensi data, sehingga kita dapat mempertahankan sebanyak mungkin informasi yang dapat dijelaskan oleh sejumlah komponen utama atau principal components.

1. **Hitung Matriks Kovarian:**<br>

   $ \text{Cov}(\mathbf{X}) = \frac{1}{n-1} \sum_{i=1}^{n} (\mathbf{X}_i - \bar{\mathbf{X}})(\mathbf{X}_i - \bar{\mathbf{X}})^T $

    di mana:


  *   $\mathbf{X}_i \text{ adalah vektor data poin ke-i,}$
  *   $\bar{\mathbf{X}} \text{ adalah vektor rata-rata dari semua data,}$
  *   $n \text{ adalah jumlah data poin.} $

2. **Hitung Nilai dan Vektor Eigen:** <br>
Menghitung nilai eigen (λ) dan vektor eigen (v) dari matriks kovarian.<br>

   $ \text{Cov}(\mathbf{X})\mathbf{v} = \lambda\mathbf{v} $

3. **Pilih Principal Components:**<br> Principal components dipilih berdasarkan nilai eigen tertinggi. Misalnya, jika kita ingin mereduksi dimensi menjadi k, kita memilih k vektor eigen dengan nilai eigen tertinggi.<br>

4. **Transformasi Data:** <br> Data asli (X) dapat diproyeksikan ke dalam ruang yang baru (Y) yang terdiri dari principal components yang dipilih.<br>

   $ \mathbf{Y} = \mathbf{X} \mathbf{V} $

    di mana:

*   (Y) : matriks yang berisi data yang telah diproyeksikan ke dalam ruang principal components,<br>
*   (X) : matriks data asli,<br>
*   (V) : matriks yang berisi vektor eigen sebagai kolom-kolomnya.<br>




In [ ]:
from sklearn.decomposition import PCA as sklearnPCA

# Membuat objek PCA dengan 9 komponen utama
sklearn_pca = sklearnPCA(n_components=9)

# Melakukan transformasi PCA pada data latih yang telah dinormalisasi
X_train_pca = sklearn_pca.fit_transform(X_train_scaled)

# Menampilkan tipe data dari X_train_pca
type(X_train_pca)

numpy.ndarray

In [ ]:
dump(sklearn_pca, open('PCAknn9.pkl', 'wb'))

In [ ]:
import pickle
with open('PCAknn9.pkl', 'rb') as pca:
   loadpca= pickle.load(pca)

In [ ]:
# Melakukan transformasi PCA pada data uji yang telah dinormalisasi menggunakan model PCA yang telah dimuat
X_test_pca = loadpca.transform(X_test_scaled)

# Menampilkan bentuk (shape) dari X_test_pca
X_test_pca.shape


(562, 9)

In [ ]:
clfr = KNeighborsClassifier(n_neighbors= 13, metric = "euclidean")
# Melatih model menggunakan data latih yang telah mengalami reduksi dimensi PCA
clfr.fit(X_train_pca, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=13)

In [ ]:
# Melakukan prediksi pada data uji yang telah mengalami reduksi dimensi PCA menggunakan model RandomForestClassifier
y_prediksi = clfr.predict(X_test_pca)

# Menampilkan hasil prediksi
y_prediksi


array(['YAF_fear', 'YAF_pleasant_surprised', 'YAF_fear', 'YAF_angry',
       'YAF_angry', 'OAF_disgust', 'OAF_disgust', 'YAF_neutral',
       'OAF_disgust', 'OAF_neutral', 'YAF_fear', 'YAF_happy',
       'YAF_pleasant_surprised', 'OAF_happy', 'YAF_pleasant_surprised',
       'YAF_pleasant_surprised', 'OAF_fear', 'YAF_sad', 'YAF_angry',
       'YAF_pleasant_surprised', 'YAF_disgust', 'OAF_neutral',
       'YAF_neutral', 'OAF_pleasant_surprise', 'YAF_sad', 'OAF_disgust',
       'OAF_disgust', 'OAF_angry', 'YAF_pleasant_surprised', 'YAF_angry',
       'YAF_happy', 'OAF_happy', 'OAF_sad', 'OAF_fear', 'YAF_disgust',
       'OAF_happy', 'OAF_disgust', 'OAF_neutral',
       'YAF_pleasant_surprised', 'YAF_pleasant_surprised', 'OAF_sad',
       'OAF_happy', 'YAF_neutral', 'OAF_fear', 'OAF_fear', 'YAF_disgust',
       'YAF_sad', 'OAF_pleasant_surprise', 'YAF_disgust', 'OAF_disgust',
       'OAF_sad', 'OAF_happy', 'YAF_disgust', 'YAF_fear', 'YAF_disgust',
       'OAF_neutral', 'OAF_angry', 'YAF_n

In [ ]:
acc_pca= accuracy_score(y_test,y_prediksi)
print("Akurasi:",acc_pca)

Akurasi: 0.7206405693950177


In [ ]:
dump(clfr, open('knnpca9fix.pkl', 'wb'))

# **APLIKASI STREAMLIT**

https://luluatulmaknunahutspsd.streamlit.app/